In [1]:
pip install ragas datasets evaluate langchain faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset
import json

ruta_dataset = r"C:\Users\bakug\OneDrive\Documentos\Metricas\dev-v2.0.json"

with open(ruta_dataset, 'r', encoding='utf-8') as f:
    squad_data = json.load(f)

data_list = []
for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answers = [ans['text'] for ans in qa['answers']]
            data_list.append({
                "question": question,
                "context": context,
                "answers": answers
            })

print(f"Total ejemplos: {len(data_list)}")

Total ejemplos: 11873


In [3]:
import random
sample_data = random.sample(data_list, 100)

In [5]:
import pandas as pd

filtered_data = [item for item in sample_data if len(item["answers"]) > 0]

df = pd.DataFrame([{
    "question": item["question"],
    "ground_truth": item["answers"][0],
    "contexts": [item["context"]],
    "answer": item["answers"][0]
} for item in filtered_data])

print(f"Ejemplos usados: {len(df)}")
print(df.head())

Ejemplos usados: 55
                                            question        ground_truth  \
0                   What was the topic of the error?  Himalayan glaciers   
1  What is the expression used to denote a worst ...               O(n2)   
2  What chemist managed to make enough liquid oxy...         James Dewar   
3  Along with English and mathematics, what subje...     natural science   
4  What is the most important type of Norman art ...             mosaics   

                                            contexts              answer  
0  [Former IPCC chairman Robert Watson has said "...  Himalayan glaciers  
1  [For example, consider the deterministic sorti...               O(n2)  
2  [In 1891 Scottish chemist James Dewar was able...         James Dewar  
3  [In the Philippines, the private sector has be...     natural science  
4  [In Britain, Norman art primarily survives as ...             mosaics  


In [7]:
pip install ragas


Note: you may need to restart the kernel to use updated packages.


In [13]:
import ragas
print(ragas.__version__)

0.3.2


In [17]:
from datasets import load_dataset, Dataset
from ragas.metrics import context_precision, faithfulness, answer_correctness
from ragas.evaluation import evaluate
import pandas as pd

dataset = load_dataset("squad")
sample_data = dataset["validation"].select(range(50)) 

data = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": []
}

for item in sample_data:
    data["question"].append(item["question"])
    data["ground_truth"].append(item["answers"]["text"][0] if item["answers"]["text"] else "")
    data["contexts"].append([item["context"]])
    data["answer"].append(item["answers"]["text"][0] if item["answers"]["text"] else "")

ragas_dataset = Dataset.from_dict(data)

result = evaluate(
    dataset=ragas_dataset,
    metrics=[context_precision, faithfulness, answer_correctness]
)

print(result)

README.md: 0.00B [00:00, ?B/s]

C:\Users\bakug\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bakug\.cache\huggingface\hub\datasets--squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [11]:
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

from datasets import load_dataset, Dataset
import pandas as pd

from ragas.metrics import LLMContextRelevance, LLMAnswerRelevance, LLMFaithfulness
from ragas.evaluation import evaluate

dataset = load_dataset("squad", split="validation")

sample_data = dataset.shuffle(seed=42).select(range(50))

df = pd.DataFrame([{
    "question": item["question"],
    "ground_truth": item["answers"]["text"][0] if len(item["answers"]["text"]) > 0 else "",
    "contexts": [item["context"]],
    "answer": item["answers"]["text"][0] if len(item["answers"]["text"]) > 0 else ""
} for item in sample_data])

print("Ejemplo de dataset preparado:")
print(df.head())

ragas_dataset = Dataset.from_pandas(df)

metrics = [
    LLMContextRelevance(),
    LLMAnswerRelevance(),
    LLMFaithfulness()
]

result = evaluate(ragas_dataset, metrics=metrics)

print("\nResultados de la evaluación:")
print(result)

ImportError: cannot import name 'LLMContextRelevance' from 'ragas.metrics' (C:\Users\bakug\anaconda3\Lib\site-packages\ragas\metrics\__init__.py)